In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import ydata_profiling

%matplotlib inline

In [ ]:
# Load the dataset
df = pd.read_excel(r'C:\Users\sg_cl\Desktop\masters subjects\DSRT 736\Fraud Dataset\New_data_CCFD\ccfraudTrain.xlsx')

df1 = pd.read_excel(r'C:\Users\sg_cl\Desktop\masters subjects\DSRT 736\Fraud Dataset\New_data_CCFD\ccfraudTest.xlsx')
# For snakey diagram
#df1 = pd.read_excel(r'C:\Users\sg_cl\Desktop\masters subjects\DSRT 736\Fraud Dataset\New_data_CCFD\Sankey_data.xlsx')mk

In [ ]:
df.columns

In [ ]:
print("Missing values:", df.isnull().sum())

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# checking the dataset
df.info()
df.describe()
df1.info()
df1.describe()

In [ ]:
#print (df['trans_date_trans_time'].head())
from datetime import datetime as dt
import pytz  # Import the pytz module for time zones

df['trans_date_trans_time_HR'] = df['trans_date_trans_time'].dt.strftime("%H")

# Convert the Unix time values to datetime objects
df['unix_time_NEW'] = df['unix_time'].apply(lambda x: dt.fromtimestamp(x))

# Set the time zone to the desired time zone (e.g., 'UTC' or your local time zone)
local_time_zone = 'America/New_York'
df['unix_time_NEW'] = df['unix_time_NEW'].dt.tz_localize(pytz.utc).dt.tz_convert(local_time_zone)


print (df.head())

df1['trans_date_trans_time_HR'] = df1['trans_date_trans_time'].dt.strftime("%H")

# Convert the Unix time values to datetime objects
df1['unix_time_NEW'] = df1['unix_time'].apply(lambda x: dt.fromtimestamp(x))

# Set the time zone to the desired time zone (e.g., 'UTC' or your local time zone)
local_time_zone = 'America/New_York'
df1['unix_time_NEW'] = df1['unix_time_NEW'].dt.tz_localize(pytz.utc).dt.tz_convert(local_time_zone)


print (df1.head())

In [ ]:
df = df.drop('unix_time', axis=1)
df = df.drop('unix_time_NEW', axis=1)
df1 = df1.drop('unix_time', axis=1)
df1 = df1.drop('unix_time_NEW', axis=1)
#df = df.drop('Errors?', axis=1)
#df = df.drop('Time', axis=1)


In [ ]:
print (df.head())

In [ ]:
print(df.corr())
df.columns

In [ ]:
profile = pandas_profiling.ProfileReport(df)
df['trans_date_trans_time_HR'].unique()

In [ ]:
profile.to_file("data_profile_report.html")

In [ ]:
df.columns

In [ ]:
# Sample dataset 1
# Convert 'Timestamp' column to datetime type
#main_df['Timestamp'] = pd.to_datetime(main_df['Timestamp'])

# Select the desired columns and perform aggregation on the subset
subset_df = df[[ 'cc_num', 'trans_date_trans_time_HR', 'merchant', 'category', 'gender','state','lat', 'long','merch_lat',
       'merch_long', 'amt','is_fraud']].reset_index()
subset_df1 = df1[[ 'cc_num', 'trans_date_trans_time_HR', 'merchant', 'category', 'gender','state','lat', 'long','merch_lat',
       'merch_long', 'amt','is_fraud']].reset_index()

In [ ]:
subset_df.head()

In [ ]:
# Convert categorical variables to numerical using one-hot encoding
subset_df = pd.get_dummies(subset_df, columns=['merchant', 'category', 'gender', 'state'])

subset_df1 = pd.get_dummies(subset_df1, columns=['merchant', 'category', 'gender', 'state'])

In [ ]:
#subset_df.head()
subset_df = subset_df.drop('state_DE', axis=1)
#subset_df1.head()
#subset_df1['state_DE'] = 0

In [ ]:
#Data pre processing String values to Int values
print(df['Is Fraud?'].unique())
y = df['Is Fraud?']
enc = preprocessing.LabelEncoder()
enc.fit(y)
y = enc.transform(y)
print(y)


# Separate features (X) and labels (y)
X = df.drop('Is Fraud?', axis=1)
print(X.head())
print(y)

In [ ]:
# Split data into features (X) and target (y)
X = subset_df.drop('is_fraud', axis=1)
y = subset_df['is_fraud']

X1 = subset_df1.drop('is_fraud', axis=1)
y1 = subset_df1['is_fraud']

In [ ]:
print("Missing values:", df.isnull().sum())

In [ ]:
# Split the dataset into training, validation, and test sets
X_train,X_test, y_train,y_test = train_test_split(X, y,test_size=0.2, stratify=y, random_state=42)
#X_val,X_test,y_val, y_test = train_test_split(X1, y1,test_size=0.5, stratify=y1, random_state=42)
#X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=42)

In [ ]:
# Print the shapes of the resulting subsets
print("Training set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)

In [ ]:
# Create a linear regression model
# = LinearRegression()
# Define class weights (adjust these based on your problem)
genuine_weight = 1.0 #6006 / (1042569 + 6006)
fraud_weight = 9.0 #1042569 / (1042569 + 6006)

class_weights = {0: genuine_weight, 1: fraud_weight}
#class_weights = {0: 1.0, 1: 10.0}


model = RandomForestClassifier(n_estimators=100,class_weight=class_weights, random_state=42,n_jobs=-1,max_samples=0.8)
model1 = RandomForestClassifier(n_estimators=30,class_weight=class_weights, random_state=42,n_jobs=-1,max_samples=0.8)
model2 = RandomForestClassifier(n_estimators=70,class_weight=class_weights, random_state=42,n_jobs=-1,max_samples=0.8)
model3 = RandomForestClassifier(n_estimators=80,class_weight=class_weights, random_state=42,n_jobs=-1,max_samples=0.8)
model4 = RandomForestClassifier(n_estimators=90,class_weight=class_weights, random_state=42,n_jobs=-1,max_samples=0.8)

In [ ]:
X_train.columns

In [ ]:
cols_to_exclude = 'is_fraud'
X_train = subset_df.drop(columns = [cols_to_exclude])
y_train = subset_df[['is_fraud']]
X_test = subset_df1.drop(columns = [cols_to_exclude])
y_test = subset_df1[['is_fraud']]

In [ ]:
# Train the model
model.fit(X_train, y_train)

# Train the model
model1.fit(X_train, y_train)
# Train the model
model2.fit(X_train, y_train)


model3.fit(X_train, y_train)

model4.fit(X_train, y_train)

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Make predictions on the test set
y1_pred = model1.predict(X_test)

# Make predictions on the test set
y2_pred = model2.predict(X_test)

# Make predictions on the test set
y3_pred = model3.predict(X_test)

# Make predictions on the test set
y4_pred = model4.predict(X_test)

In [ ]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y1_pred))
print("\nClassification Report:\n", classification_report(y_test, y1_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y2_pred))
print("\nClassification Report:\n", classification_report(y_test, y2_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y3_pred))
print("\nClassification Report:\n", classification_report(y_test, y3_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y4_pred))
print("\nClassification Report:\n", classification_report(y_test, y4_pred))

In [ ]:
 # Replace clf with your Random Forest model and X_test with your test data
predicted_probabilities = model.predict_proba(X_test)[:, 1] 
predicted_probabilities1 = model1.predict_proba(X_test)[:, 1] 
predicted_probabilities2 = model2.predict_proba(X_test)[:, 1] 
predicted_probabilities3 = model3.predict_proba(X_test)[:, 1] 
predicted_probabilities4 = model4.predict_proba(X_test)[:, 1] 

In [ ]:
predicted_probabilities4

In [ ]:
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]  # Example threshold values
for threshold in thresholds:
    y_pred = (predicted_probabilities >= threshold).astype(int)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"Threshold: {threshold:.2f} | Precision: {precision:.2f} | Recall: {recall:.2f} | F1-score: {f1:.2f}")

In [ ]:
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]  # Example threshold values
for threshold in thresholds:
    y1_pred = (predicted_probabilities1 >= threshold).astype(int)
    precision = precision_score(y_test, y1_pred)
    recall = recall_score(y_test, y1_pred)
    f1 = f1_score(y_test, y1_pred)
    print(f"Threshold: {threshold:.2f} | Precision: {precision:.2f} | Recall: {recall:.2f} | F1-score: {f1:.2f}")

In [ ]:
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]  # Example threshold values
for threshold in thresholds:
    y2_pred = (predicted_probabilities2 >= threshold).astype(int)
    precision = precision_score(y_test, y2_pred)
    recall = recall_score(y_test, y2_pred)
    f1 = f1_score(y_test, y2_pred)
    print(f"Threshold: {threshold:.2f} | Precision: {precision:.2f} | Recall: {recall:.2f} | F1-score: {f1:.2f}")

In [ ]:
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]  # Example threshold values
for threshold in thresholds:
    y3_pred = (predicted_probabilities3 >= threshold).astype(int)
    precision = precision_score(y_test, y3_pred)
    recall = recall_score(y_test, y3_pred)
    f1 = f1_score(y_test, y3_pred)
    print(f"Threshold: {threshold:.2f} | Precision: {precision:.2f} | Recall: {recall:.2f} | F1-score: {f1:.2f}")

In [ ]:
thresholds = [0.2] 
#thresholds = [0.2,0.3, 0.4, 0.5, 0.6, 0.7,0.8]  # Example threshold values
for threshold in thresholds:
    y4_pred = (predicted_probabilities4 >= threshold).astype(int)
    precision = precision_score(y_test, y4_pred)
    recall = recall_score(y_test, y4_pred)
    f1 = f1_score(y_test, y4_pred)
    print(f"Threshold: {threshold:.2f} | Precision: {precision:.2f} | Recall: {recall:.2f} | F1-score: {f1:.2f}")
    print("Confusion Matrix:\n", confusion_matrix(y_test, y4_pred))
    print("\nClassification Report:\n", classification_report(y_test, y4_pred))

In [ ]:
# Split the dataset into training, validation, and test sets
#X_train,X_test, y_train,y_test = train_test_split(X, y,test_size=0.2, stratify=y, random_state=42)
X_val,X_test,y_val, y_test = train_test_split(X1, y1,test_size=0.5, stratify=y1, random_state=42)

In [ ]:
# Make predictions on the test set
y4_pred = model4.predict(X_test)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
thresholds = [0.3] 
#thresholds = [0.2,0.3, 0.4, 0.5, 0.6, 0.7,0.8]  # Example threshold values
for threshold in thresholds:
    y4_pred = (predicted_probabilities4 >= threshold).astype(int)
    precision = precision_score(y_test, y4_pred)
    recall = recall_score(y_test, y4_pred)
    f1 = f1_score(y_test, y4_pred)
    print(f"Threshold: {threshold:.2f} | Precision: {precision:.2f} | Recall: {recall:.2f} | F1-score: {f1:.2f}")
    print("Confusion Matrix:\n", confusion_matrix(y_test, y4_pred))
    print("\nClassification Report:\n", classification_report(y_test, y4_pred))

In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

precision, recall, thresholds = precision_recall_curve(y_test, predicted_probabilities4)
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()

In [ ]:
# Add the predicted labels to the dataset
X_test['predicted_label'] = y_pred

In [ ]:
y.columns


In [ ]:
category_totals = df.groupby('is_fraud')['is_fraud'].count()
print(category_totals)

In [ ]:
X_test.columns

In [ ]:
# Identify false negative transactions
false_negatives = X_test[(X_test['predicted_label'] == 0) & (X_test['is_fraud'] == 1)]
print(false_negatives)

In [ ]:
print(y_pred)
# Set a threshold
threshold = 0.01

# Convert predictions to binary classes based on the threshold
y_pred_binary = [1 if val >= threshold else 0 for val in y_pred]

# Print the binary predictions
print("Binary Predictions:", y_pred_binary)

In [ ]:
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)

In [ ]:
# Select an example for validation (optional)
example_index = 208
example_features = X.iloc[example_index]
example_target = y[example_index]

# Make predictions on the example
example_prediction = model.predict([example_features])

# Compare with actual value
print("Example features:", example_features)
print("Example target:", example_target)
print("Example prediction:", example_prediction)

# Calculate mean squared error on the test set (optional)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print(y_pred)

In [ ]:
counts = y_pred_binary.value_counts()